Write the QOnuchic output during an OpenSMOG simulation.

REQUIRED: python package OpenMM and [OpenSMOG](https://github.com/smog-server/OpenSMOG.git)

In [ ]:
import sys
import os
import re as regex

In [2]:
from OpenSMOG import SBM

    ****************************************************************************************    
       **** *** *** *** *** *** *** *** OpenSMOG (v1.2) *** *** *** *** *** *** *** ****        

           The OpenSMOG class is used to perform molecular dynamics simulations using           
                     Structure-Based Models (SBM) for biomolecular systems,                     
             and it allows for the simulation of a wide variety of potential forms.             
                      OpenSMOG uses force field files generated by SMOG 2.                      
                             OpenSMOG documentation is available at                             
                  https://opensmog.readthedocs.io and https://smog-server.org                   

                    OpenSMOG is described in: Oliveira and Contessoto et al,                    
              SMOG 2 and OpenSMOG: Extending the limits of structure-based models.              
                    Protein 

In [ ]:
sys.path.append(os.path.abspath(".."))
from Contacts import *

In [ ]:
def add_QOnuchicReporter(sbm, reportInterval, cutoff = 1.5, Qi = False, QName = None):
    R"""Add a QOnuchicReporter to the SBM object, modified from several OpenSMOG functions by Atonio Oliveira

         Arg:

            sbm (SBM, required):
                A SBM object from the OpenSMOG package.
            reportInterval (int, required):
                The interval to report the Q value.
            cutoff (float, optional):
                The cutoff distance. Within cutoff*original_distance, the contact is considered formed.
            Qi (bool, optional):
                If True, the formation of every single contacts (Qi) is calculated.
            QName (str, optional):
                The name of the file to save the Q values. If None, the file will be named as the SBM object name + '_q.txt'.
        """
    if QName is None:
        qfile = os.path.join(sbm.folder, sbm.name+ '_q.txt')
    else:
        if os.path.basename(QName) != QName:
            sbm.opensmog_quit('qName is invalid. To specify the path, use the saveFolder method.')
        if regex.search(".txt$",QName):
            qfile = os.path.join(sbm.folder, QName)
        else:
            qfile = os.path.join(sbm.folder, QName + ".txt")

    if os.path.isfile(qfile):
        i = 1
        ck = True
        while i <= 10 and ck:
            root, ext = os.path.splitext(qfile)
            newname = f"{root}_{i}{ext}"
            if not os.path.isfile(newname):
                print(f"{qfile} already exists.  Backing up to {newname}")
                os.rename(qfile, newname)
                ck = False
            else:
                i += 1

    forces = [sbm.forcesDict[i] for i in sbm.contacts]
    sbm_contacts = ContactsOnuchic()
    sbm_contacts.loadContacts(openmm_forces = forces)
    sbm.simulation.reporters.append(QOnuchicReporter(qfile, reportInterval, sbm_contacts, cutoff=cutoff, Qi=Qi, step=True))
    return sbm_contacts

Load the SBM

In [5]:
sbm_AA = SBM(name='3ski', time_step=0.002, collision_rate=1.0, r_cutoff=0.65, temperature=0.6, cmm=False, pbc=False)
sbm_AA.setup_openmm(platform='CPU')
sbm_AA.saveFolder('output')
sbm_AA_grofile = 'SBM/3skict.gro'
sbm_AA_topfile = 'SBM/3skict.top'
sbm_AA_xmlfile = 'SBM/3skict.xml'

sbm_AA.loadSystem(Grofile=sbm_AA_grofile, Topfile=sbm_AA_topfile, Xmlfile=sbm_AA_xmlfile)
sbm_AA.createSimulation()
sbm_AA.minimize()
sbm_AA.createReporters(trajectory=True, interval=10)

Will try to load coordinates from SBM/3skict.gro
Will try to load force field from SBM/3skict.top
This simulation will not use Periodic boundary conditions
Will try to load OpenSMOG-specific force field terms from SBM/3skict.xml


Contacts found in the xml file.  Will include definitions
provided in the top and xml files.


Nonbonded parameters not found in XML file.  Will
only use information nonbonded parameters that
are provided in the top file.


Dihedral definitions found in XML file. Will include 
dihedral information provided in the top and xml files.

        Creating Contacts force contact_1-6-12 from xml file
        Creating Dihedrals force dihedral_harmonic from xml file
        Creating Dihedrals force dihedral_ncos from xml file
Loaded force field and config files.
Creating the simulation with the following parameters:
        name : 3ski
        platform : CPU
        precision : platform default
        integrator : LangevinMiddleIntegrator
        timestep : 0.002
    

Add the reporter and return the Contact used in the simulation

In [6]:
sbm_contacts = add_QOnuchicReporter(sbm_AA, 10, Qi=True)

In [ ]:
# Run the simulation
# sbm_AA.run(1000)